In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
from tqdm import tqdm
from random import randint, choice, uniform
from math import ceil
from statistics import stdev, mean
from skopt import gbrt_minimize, dummy_minimize, forest_minimize, gp_minimize
from qiskit import *
from qiskit.visualization import plot_histogram
provider = IBMQ.load_account()

class Graph():
    def __init__(self, N, randomize=True):
        ''' Initialize a random graph with N vertices. '''
        self.N = N
        self.E = 0
        self.adj = {n:dict() for n in range(N)}
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []
        if randomize:
            self.randomize()

    def randomize(self):
        ''' Randomly generate edges for this graph. '''

        # Generate list of tuples for all possible directed edges.
        all_possible_edges = set([(x,y) for x in range(self.N) for y in range(self.N) if x != y])

        # Sanity check, ensuring we generated the correct number of edges.
        e_gen = len(all_possible_edges) / 2
        e_shd = self.N * (self.N-1) / 2
        assert e_gen == e_shd , "%d != %d" % (e_gen, e_shd)

        # Choose a random number of edges for this graph to have. 
        # Note, we stop at len/2 because we generated directed edges,
        # so each edge counts twice.
        num_edges = randint(1, len(all_possible_edges)/2)
        for i in range(num_edges):
            # Choose an edge, remove it and its directed complement from the list.
            e = choice(list(all_possible_edges))
            all_possible_edges.remove(e)
            all_possible_edges.remove(e[::-1])

            # Unpack tuple into vertex ints.
            u, v = int(e[0]), int(e[1])

            # Choose a random weight for each edge.
            weight = randint(1, 100)

            #weight = 1
            self.add_edge(u, v, weight)


    def add_edge(self, u, v, weight):
        ''' Add an edge to the graph. '''
        self.E += 1
        self.adj[u][v] = weight

    def get_edges(self):
        ''' Get a list of all edges. '''
        edges = []
        for u in self.adj:
            for v in self.adj[u]:
                edges.append((u, v, self.adj[u][v]))
        return edges

    def get_score(self,bitstring):
        ''' Score a candidate solution. '''
        assert len(bitstring) == self.N

        score = 0

        # For every edge u,v in the graph, add the weight
        # of the edge if u,v belong to different cuts
        # given this canddiate solution.

        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    score += self.adj[u][v]
        return score

    def optimal_score(self):
        '''
        Returns (score, solutions) holding the best possible solution to the
        MaxCut problem with this graph.
        '''

        best = 0
        best_val = []

        # Iterate over all possible candidate bitstrings
        # Note: the bitstrings from 0 - N/2 are symmetrically
        # equivalent to those above
        for i in range(ceil((2 ** self.N)/2)):
            # Convert number to 0-padded bitstring.
            bitstring = bin(i)[2:]
            bitstring = (self.N - len(bitstring)) * "0" + bitstring

            sc = self.get_score(bitstring)
            if sc > best:
                best = sc
                best_val = [bitstring]
            elif sc == best:
                best_val.append(bitstring)
        return best, best_val

    def edges_cut(self, bitstring):
        ''' Given a candidate solution, return the number of edges that this solution cuts. '''
        num = 0
        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    num += 1
        return num

    def update_score(self, bitstring):
        ''' Scores the given bitstring and keeps track of best. '''
        score = self.get_score(bitstring)
        if score > self.currentScore:
            self.currentScore = score
            self.currentBest = bitstring
            print(self.currentBest)
        return score
    
    def clear_runs(self):
        ''' Clear data from past runs. '''
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []
        
    def add_run(self, gamma, beta, expected_value):
        ''' Save the data from each run iteration. '''
        self.runs.append([gamma, beta, expected_value])
        
    def __str__(self):
        return "Graph with %d vertices %d edges.\nAdjacency List: %s" % (self.N, self.E, self.adj)

def get_expectation(x, g, NUM_SHOTS=1024):
    gamma, beta = x
    print("Cost of Gamma: %s, beta: %s... " % (gamma, beta))
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)
    for i in range(g.N):
        qc.h(q[i])
    for edge in g.get_edges():
        u, v, w = edge
        qc.cx(q[u], q[v])
        qc.u1(gamma*w, q[v])
        qc.cx(q[u], q[v])
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])
    for i in range(g.N):
        qc.measure(q[i], c[i])
    qc.draw()
    backend = BasicAer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=NUM_SHOTS)
    results = job.result()
    result_dict = results.get_counts(qc)
    print("done!\n")

    # Calculate the expected value of the candidate bitstrings.
    exp = 0
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / NUM_SHOTS
        score = g.update_score(bitstring)

        # Expected value is the score of each bitstring times
        # probability of it occuring.
        exp += score * prob

    print("\tExpected Value: %s\n" % (exp))
    g.add_run(gamma, beta, exp)

    return exp

def hold_constant(vary="gamma"):
    ''' Plots expected value vs. gamma/beta, holding the rest of the variables constant.'''
    # Choose some random starting beta/gamma and graph.
    lim = np.pi if vary == "gamma" else 2*np.pi
    constant_var = uniform(0, lim)
    g = Graph(13)
    print(g)
    # RUNS # of runs at each gamma for error bars.
    RUNS = 3

    # Keep track of gammas, expected values, for plotting.
    pts, exp, std = [], [], []

    # The maximum possible expected value is the maximum possible weighted cut.
    opt = g.optimal_score()[0]
    print("Running hold_constant!\n")
    print("Optimal score: %s\n" % (opt))
    
    # Number of data points to collect.
    NUM_RUNS = 60
    MIN = 0
    MAX = 2*np.pi if vary == "gamma" else np.pi
    
    
    points = np.linspace(MIN, MAX, NUM_RUNS)
    for point in points:
        pts.append(point)

        # Calculate expected values.
        vals = []
        for i in range(RUNS):
            # Params are passed in as gamma, beta, so order matters.
            params = [point, constant_var] if vary == "gamma" else [constant_var, point]
            vals.append(get_expectation(params, g))

        # Calculate mean, standard deviation.
        exp.append(mean(vals))
        std.append(stdev(vals))

    print("Best Cut : " + g.currentBest)
    fig, ax = plt.subplots()

    ax.errorbar(x=pts, y=exp, yerr=std, fmt='o-', markersize=10)
    ax.legend(loc=2)

    # Names for plotting.
    vary_name = "Gamma" if vary == "gamma" else "Beta"
    const_name = "Beta" if vary_name == "Gamma" else "Gamma"
    
    ax.set_title("Effect of Varying %s with %s = %s" % (vary_name, const_name, constant_var))
    ax.set_xlabel("%s" % (vary_name)) 
    ax.set_ylabel("Expected Value")
    plt.show()
    
hold_constant()

Graph with 13 vertices 27 edges.
Adjacency List: {0: {11: 66, 4: 74}, 1: {11: 5, 4: 19}, 2: {6: 98, 11: 98}, 3: {2: 100}, 4: {2: 19}, 5: {7: 4}, 6: {1: 25}, 7: {2: 64, 4: 75}, 8: {1: 92}, 9: {2: 97, 11: 11, 8: 23, 6: 46, 0: 48, 4: 80, 7: 64}, 10: {12: 37, 7: 6, 0: 23}, 11: {12: 85, 8: 49}, 12: {5: 29, 6: 55}}
Running hold_constant!

Optimal score: 1142

Cost of Gamma: 0.0, beta: 3.136983929194633... 
done!

1011001101111
0011011000010
0010100010011
1010010010001
1010110110001
1110000001101
1110010100001
0101001101110
	Expected Value: 698.154296875

Cost of Gamma: 0.0, beta: 3.136983929194633... 
done!

	Expected Value: 698.154296875

Cost of Gamma: 0.0, beta: 3.136983929194633... 
done!

	Expected Value: 698.154296875

Cost of Gamma: 0.10649466622338281, beta: 3.136983929194633... 
done!

0101011101110
	Expected Value: 694.599609375

Cost of Gamma: 0.10649466622338281, beta: 3.136983929194633... 
done!

	Expected Value: 694.599609375

Cost of Gamma: 0.10649466622338281, beta: 3.1369839

done!

	Expected Value: 697.4677734375

Cost of Gamma: 2.66236665558457, beta: 3.136983929194633... 
done!

	Expected Value: 697.4677734375

Cost of Gamma: 2.768861321807953, beta: 3.136983929194633... 
done!

	Expected Value: 695.884765625

Cost of Gamma: 2.768861321807953, beta: 3.136983929194633... 
done!

	Expected Value: 695.884765625

Cost of Gamma: 2.768861321807953, beta: 3.136983929194633... 
done!

	Expected Value: 691.986328125

Cost of Gamma: 2.875355988031336, beta: 3.136983929194633... 
done!

	Expected Value: 692.845703125

Cost of Gamma: 2.875355988031336, beta: 3.136983929194633... 
done!

	Expected Value: 692.845703125

Cost of Gamma: 2.875355988031336, beta: 3.136983929194633... 
done!

	Expected Value: 692.845703125

Cost of Gamma: 2.981850654254719, beta: 3.136983929194633... 
done!

	Expected Value: 695.0087890625

Cost of Gamma: 2.981850654254719, beta: 3.136983929194633... 
done!

	Expected Value: 695.0087890625

Cost of Gamma: 2.981850654254719, beta: 3.1369839

done!

	Expected Value: 694.873046875

Cost of Gamma: 5.537722643615906, beta: 3.136983929194633... 
done!

	Expected Value: 694.873046875

Cost of Gamma: 5.644217309839289, beta: 3.136983929194633... 
done!

	Expected Value: 696.642578125

Cost of Gamma: 5.644217309839289, beta: 3.136983929194633... 
done!

	Expected Value: 695.7080078125

Cost of Gamma: 5.644217309839289, beta: 3.136983929194633... 
done!

	Expected Value: 695.7080078125

Cost of Gamma: 5.750711976062672, beta: 3.136983929194633... 
done!

	Expected Value: 696.42578125

Cost of Gamma: 5.750711976062672, beta: 3.136983929194633... 
done!

	Expected Value: 696.42578125

Cost of Gamma: 5.750711976062672, beta: 3.136983929194633... 
